In [1]:
import datetime
import numpy as np
from matplotlib import cm, pyplot as plt
import pandas as pd
from pandas_datareader import data
import ta
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score

In [2]:
todays_date = datetime.datetime.today()
start_date = todays_date - datetime.timedelta(days=2*365)
todays_date = todays_date.strftime("%Y-%m-%d")
start_date = start_date.strftime("%Y-%m-%d")
#df = data.DataReader("GAIL.NS",start='2017-01-01',end='2021-05-04', data_source='yahoo')
print(todays_date)
print(start_date)
df = data.DataReader("GAIL.NS",start=start_date,end=todays_date, data_source='yahoo')
df.to_csv("./static/tcs_ns.csv")

2021-06-11
2019-06-12


In [3]:
df=pd.read_csv("./static/tcs_ns.csv")

In [4]:
df["ho"]=(((df.High-df.Open)/df.Open)*100)
df["ol"]=(((df.Open-df.Low)/df.Open)*100)
df["oc"]=(((df.Open-df.Close)/df.Open)*100)
df["hl"]=(((df.High-df.Low)/df.Low)*100)
df['rsi']=ta.momentum.RSIIndicator(df["Close"], window=14, fillna=False).rsi()
df['r']=ta.momentum.WilliamsRIndicator(df["High"], df["Low"], df["Close"], lbp=14, fillna=False).williams_r()
df['cci']=ta.trend.CCIIndicator(df["High"], df["Low"], df["Close"], window=20, constant=0.015, fillna=False).cci()
df["m1"]=(df["Close"]/df["Close"].shift(1))*100
df["m2"]=(df["Close"]/df["Close"].shift(2))*100
df["m3"]=(df["Close"]/df["Close"].shift(3))*100
df["pvc"]=(df["Volume"]-df["Volume"].shift(1))/df["Volume"].shift(1)*100
df["pdc"]=(df["Close"]-df["Close"].shift(1))/df["Close"].shift(1)*100
df['std_3'] = df['pdc'].rolling(5).std()
df['ret_3'] = df['pdc'].rolling(5).mean()
# df["nextOpen"]=(df["Open"].shift(-1)-df["Close"])/df["Close"]*100
df["nextOpen"]=(df["Open"]-df["Close"].shift(1))/df["Close"].shift(1)*100
df["oc1"]=df["oc"].shift(-1)
df["pdc1"]=df["pdc"].shift(-1)
df["Signal"]=np.where(((df['pdc1'] >= 0.5)),"BUY",(np.where(((df['pdc1'] <= -1)),"SELL","HOLD")))
df["Signal"].value_counts()
X=df[['ho','ol','oc','hl','pdc','m1','m2','m3','rsi','r','cci','nextOpen']]
Y=df[['Signal']]

In [5]:
# len(Y)

In [6]:
# Total dataset length
dataset_length = df.shape[0]

# Training dataset length
split = int(dataset_length * 0.8)
X_train = X[21:split]
X_test = X[split:-2]
Y_train = Y[21:split]
Y_test = Y[split:-2]
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(374, 12) (97, 12)
(374, 1) (97, 1)


In [7]:
clf = RandomForestClassifier(n_estimators=100,random_state=5)
clf.fit(X_train, Y_train)

c:\users\harsh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(random_state=5)

In [8]:
# p=clf.predict(X_train)
# report = classification_report(Y_train, p)
# print(report)
# results = confusion_matrix(Y_train, p) 
# print('Confusion Matrix :')
# print(results) 
# print('Accuracy Score :',accuracy_score(Y_train, p))

In [9]:
# t=clf.predict(X_test)
# results = confusion_matrix(Y_test, t) 
# print('Confusion Matrix :')
# print(results) 
# print('Accuracy Score :',accuracy_score(Y_test, t))
# report = classification_report(Y_test, t)
# print(report)

In [10]:
n = df[['ho','ol','oc','hl','pdc','m1','m2','m3','rsi','r','cci','nextOpen']][-1:]
t1 = clf.predict(n)
print('prediction')
print((t1))

prediction
['BUY']
